<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/FineTuning_LLM_Meta_Llama_3_8B_for_MEDAL_APRIL2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://llama.meta.com/docs/how-to-guides/fine-tuning/

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

!pip install -U flash-attn --no-build-isolation --quiet

! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

#! pip install trl==0.8.6 -q

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [3]:
from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [4]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from trl import SFTTrainer

In [5]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
torch.__version__

'2.6.0+cu124'

In [ ]:
!python --version
!nvcc --version
!nvidia-smi

In [ ]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

In [ ]:
from datasets import load_dataset
# train dataset
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/train_dataset.json", split="train")

#### eval dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

In [ ]:
dataset

In [ ]:
eval_dataset

In [ ]:
print(dataset[345]['text'])

In [ ]:
print(dataset[345]['label'])

In [14]:
train_question=dataset['text']
train_answer=dataset['label']

eval_question=eval_dataset['text']
eval_answer=eval_dataset['label']

In [15]:
!rm -rf /content/medal-qa-train.txt
!rm -rf /content/medal-qa-eval.txt
filename_train='/content/medal-qa-train.txt'
filename_eval='/content/medal-qa-eval.txt'

import re
from tqdm import tqdm  # Import tqdm

n_samples_train=10000
n_samples_eval=2000


# Processing train data
for n in tqdm(range(n_samples_train), desc="Processing train data"):  # Wrap range with tqdm
    if train_answer[n] == None:
       train_answer[n] = 'Not possible to get or use'
    string = train_answer[n]
    str_question=train_question[n]
    question= re.sub("\n", "", str_question)
    answer = re.sub("\[", "", str(string))
    answer = re.sub("\]", "", str(answer))
    answer0=answer[1:len(answer)-1]

    if len(answer0)==0:
        answer='Not possible to get or use'


    text='<s>[INST] %s [/INST] %s </s>'%(question,answer0)
    #print(text)
    with open(filename_train, 'a') as f:
        f.write(text + '\n')

# Processing eval data
for n in tqdm(range(n_samples_eval), desc="Processing eval data"):  # Wrap range with tqdm
    if eval_answer[n] == None:
       eval_answer[n] = 'Not possible to get or use'
    string = eval_answer[n]
    str_question=eval_question[n]
    question= re.sub("\n", "", str_question)
    answer = re.sub("\[", "", str(string))
    answer = re.sub("\]", "", str(answer))
    answer0=answer[1:len(answer)-1]

    if len(answer0)==0:
        answer='Not possible to get or use'


    text='<s>[INST] %s [/INST] %s </s>'%(question,answer0)

    with open(filename_eval, 'a') as f:
        f.write(text + '\n')


Processing eval data: 100%|██████████| 2000/2000 [00:00<00:00, 38030.98it/s]


In [16]:
train_question[6]

'to quantify the amount of CL NO harvested in modified RND'

In [17]:
train_answer[6]

['radical neck dissection']

In [ ]:
eval_question[100]

In [19]:
eval_answer[100]

['childhood brain tumors']

In [ ]:
text0_train = load_dataset("text", data_files="/content/medal-qa-train.txt", split="train")

In [ ]:
text0_train[6]

In [ ]:
text0_eval = load_dataset("text", data_files="/content/medal-qa-eval.txt", split="train")

In [ ]:
text0_eval[100]

In [24]:
nrec=n_samples_train

dataset_final_Question=dataset['text'][0:nrec]
dataset_final_Answer=dataset['label'][0:nrec]
dataset_final_text=text0_train[0:nrec]['text']

# EVAL
nrec=n_samples_eval

eval_dataset_final_Question=eval_dataset['text'][0:nrec]
eval_dataset_final_Answer=eval_dataset['label'][0:nrec]
eval_dataset_final_text=text0_eval[0:nrec]['text']

In [25]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer
datasetF['text'] = dataset_final_text

datasetF_eval = pd.DataFrame() # Create an empty DataFrame
datasetF_eval['Question'] = eval_dataset_final_Question
datasetF_eval['Answer'] = eval_dataset_final_Answer
datasetF_eval['text'] = eval_dataset_final_text

In [ ]:
datasetF

In [ ]:
datasetF['text'][6]

In [ ]:
datasetF_eval

In [ ]:
datasetF_eval['text'][100]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

model_id = "meta-llama/Meta-Llama-3-8B"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

In [ ]:
print(model)

In [32]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        #[q_proj k_proj): v_proj o_proj)
        task_type="CAUSAL_LM",
)

In [33]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="/content/gdrive/MyDrive/model/2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine",

    num_train_epochs=1,                    # number of training epochs for POC
    per_device_train_batch_size=3,          # batch size per device during training
    #2
    gradient_accumulation_steps=6,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    #gradient_checkpointing_kwargs={"use_reentrant": True},
    optim="adamw_torch_fused",              # use fused adamw optimizer

    #trainer = Trainer(model=model, args=training_args, train_dataset=ds, optimizers=(adam_bnb_optim, None))
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch

    learning_rate=2e-6,                     # learning rate, based on QLoRA paper # i used in the first model

    #learning_rate=1.41e-5,
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper = 0.03

    #lr_scheduler_type="constant",           # use constant learning rate scheduler
    lr_scheduler_type="cosine",            # lr_scheduler_type="cosine" (Cosine Annealing Learning Rate)


    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    gradient_checkpointing_kwargs={"use_reentrant": True},

    eval_strategy="steps",
    eval_steps=50,
    #force_download=True,
)

In [34]:
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint
#torch.utils.checkpoint.checkpoint(use_reentrant=False)

https://huggingface.co/docs/trl/main/en/sft_trainer

In [ ]:
from trl import SFTTrainer
import datasets

# Convert Pandas DataFrame to Hugging Face Dataset
datasetF_hf = datasets.Dataset.from_pandas(datasetF)

datasetF_hf_eval = datasets.Dataset.from_pandas(datasetF_eval)

max_seq_length = 3072 # max sequence length for model and packing of the dataset

# Explicitly set the device using torch.device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Add a padding token to your tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Or, add a new padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=datasetF_hf,
    eval_dataset=datasetF_hf_eval,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py

In [ ]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

In [37]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

In [2]:
%rm -rf /content/tensorbard
%mkdir -p /content/tensorbard
%cd /content/tensorbard
!git lfs install
!git clone https://huggingface.co/frankmorales2020/2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/content/tensorbard
Git LFS initialized.
Cloning into '2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 1), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (26/26), 8.02 KiB | 1.00 MiB/s, done.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/tensorbard/2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine/runs

Test Model and run Inference

In [ ]:
!pip install colab-env --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

!pip install -U flash-attn --no-build-isolation --quiet

! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

In [ ]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
import colab_env

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# 2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine
peft_model_id = "/content/gdrive/MyDrive/model/2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine"


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    peft_model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Let’s load our test dataset try to generate an instruction.

In [ ]:
print(model)

In [7]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
nrec= randint(0, len(eval_dataset))
nrec=600

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['text']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

Device set to use cuda:0


In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
ganswer

In [10]:
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
oanswer=str(eval_dataset[nrec]['label'])
oanswer=oanswer[2:len(oanswer)-2]
print(f"Original Answer:\n{oanswer}")
print()
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
qc=str(ganswer).find('[INST]')
ganswer=ganswer[0:qc-7]
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0]
qc=str(ganswer).find('[/INST]')
if qc>0:
  ganswer=ganswer[qc+8:len(ganswer)]
print(f"Generated Answer:\n{ganswer}")
print()
if ganswer == oanswer:
  print("Match")
else:
  print("NO Match")

Query:
the CTS rhythmically controls SMB physiology cellular proliferation and xenobiotic metabolism over the h period the SCN in the hypothalamus coordinate the molecular clocks in most rat cells through an array of circadian physiological rhythms including restactivity body temperature feeding patterns and hormonal secretions as a result shift work that involves circadian disruption is probably carcinogenic in humans in experimental models chronic jetlag cjl suppresses restactivity and body temperature rhythms and accelerates growth of two transplantable tumors in mice cjl also suppresses or significantly alters the expression rhythms of clock genes in liver and PT circadian clock disruption from cjl downregulates p and upregulates cmyc thus favoring cellular proliferation here we investigate the role of cjl as a tumor promoter in mice exposed to the hepatic carcinogen diethylnitrosamine den

Original Answer:
circadian timing system

Generated Answer:
that cjl accelerates deninduced 

In [ ]:
nrec=6
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
print(f"Original Answer:\n{eval_dataset[nrec]['label']}")
print()
print(f"Full Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
#print(qc)
if int(qc)<0:
    #print(f"Generated Answer:\n{ganswer}")
    ganswer=outputs[0]['generated_text'][len(prompt)+8:].strip()
else:
    ganswer=ganswer[qc:len(ganswer)]
ganswer